In [39]:
import numpy as np
import pandas as pd

In [40]:
'''This notebook takes in the future timetables from the Excel file provide by Caltrain,
cleans them up, and writes them to CSV'''

'This notebook takes in the future timetables from the Excel file provide by Caltrain,\ncleans them up, and writes them to CSV'

In [41]:
#This adds dummy train names
def fill_train_ids(df):
    df = df.copy()
    n = {} #For multi trains
    m = 0 #For single trains
    for i, val in df.iloc[1].iteritems():
        if i > 1:
            #Get existing train ID (may be null)
            train_id = str(df.at[1, i])
            
            #True if train goes to future
            to_future = (df.iloc[:,i].str.strip() == '↳').any()
            
            #True if train is from past
            from_past = (df.iloc[:,i].str.strip() == '↴').any()
            
            #If this is the first instance of this train:
            if to_future and not from_past:
                #Increment train ID. If it doesn't exist, create it as 1
                n[train_id] = n.get(train_id, 0) + 1
                    
            #If this is any instance of a multi train
            if to_future or from_past:
                
                #Set dummy train ID
                df.at[1, i] = train_id + '_multi_' + str(n[train_id])
                
            #This is a single train
            else:
                #Set dummy train ID
                df.at[1, i] = str(df.at[1, i]) + '_single_' + str(m)
                m += 1
    return df

In [42]:
#This sets the top two rows a column names
def set_column_names(df):
    df = df.copy()
    
    #Combine top two rows into one row
    df.loc[-1] = df.iloc[0].astype(str) + ' ' + df.iloc[1].astype(str)
    df.sort_index(inplace=True)  # sorting by index
    
    #Fix column names for variables
    df.iloc[0,0] = 'km'
    df.iloc[0,1] = 'station'
    
    #Set column names
    df = df.rename(columns=df.iloc[0]).drop(df.index[0]).reset_index(drop=True)
    
    #Drop top two rows
    df.drop(df.index[0:2], inplace=True)
    
    return df

In [43]:
#This converts the time column to minutes since midnight
def convert_times(df):
    df = df.copy()
    
    #Create hr and min columns
    df['hr'], df['min'] = df['time']\
        .str.replace('|', '')\
        .str.replace('↳', '')\
        .str.replace('↴', '')\
        .str.replace('o', '')\
        .str.strip()\
        .str.split(':').str

    #Tweak times after midnight
    df.loc[df['hr'] == '0', ['hr']] = 24
    df.loc[df['hr'] == '1', ['hr']] = 25

    #Replace blanks with NaNs so we can convert to float
    df.loc[df['hr'] == '', ['hr']] = np.NaN

    #Calculate time in minutes
    df['time'] = df['hr'].astype(float)*60 + df['min'].astype(float)
    
    #Remove rows where time column is NaN (train does not stop)
    df = df[df['time'] == df['time']].reset_index(drop=True)
    
    #Convert to ints
    df['time'] = df['time'].astype(int)
    df['hr'] = df['hr'].astype(int)
    df['min'] = df['min'].astype(int)

    return df

In [44]:
#Load all schedules and process
xls = pd.ExcelFile('../data/1 - Weekday Schedule.xlsx')

li = []

for sheet_name in xls.sheet_names:
    print('Loading {}...'.format(sheet_name))
    df = xls.parse(sheet_name, header=None, skiprows=[0,1,2,3,4,5,6])
    
    #Remove useless rows
    df.drop(df.index[2:6], inplace=True)
    
    #Remove blank column
    #df.drop(columns=[2], inplace=True)
        
    df = df.reset_index(drop=True)

    df = fill_train_ids(df)

    df = set_column_names(df)
    
    #San Jose is double-entered in some timetables; drop one of them by dropping rows where km is NaN
    df = df[(df['km'] == df['km'])]
    
    #Unpivot table
    df = df.melt(id_vars=['km', 'station'], var_name='train_id', value_name='time')
    
    df = convert_times(df)
    
    #Set scenario name
    df['scenario'] = sheet_name
    
    li.append(df)

df_final = pd.concat(li, axis=0, ignore_index=True)

Loading Baseline SB...
Loading Baseline NB...
Loading Moderate SB...
Loading Moderate NB...
Loading High SB...
Loading HIgh NB...


In [51]:
df_final.groupby('scenario').count()

,km,station,train_id,time,hr,min
scenario,,,,,,
Baseline NB,1739,1739,1739,1739,1739,1739
Baseline SB,1653,1653,1653,1653,1653,1653
HIgh NB,3432,3432,3432,3432,3432,3432
High SB,3432,3432,3432,3432,3432,3432
Moderate NB,2836,2836,2836,2836,2836,2836
Moderate SB,2836,2836,2836,2836,2836,2836


In [61]:
df_final[(df_final['scenario'] == 'High SB') & (df_final['train_id'] == 'REG (C-LCL) 101_multi_1')]

,km,station,train_id,time,hr,min,scenario
9068,0,SALESFORCE TRANSIT CENTER (STC) $,REG (C-LCL) 101_multi_1,301,5,1,High SB
9069,2,TOWNSEND / 4TH AND KING $,REG (C-LCL) 101_multi_1,306,5,6,High SB
9070,5,22nd STREET,REG (C-LCL) 101_multi_1,310,5,10,High SB
9071,11,BAYSHORE,REG (C-LCL) 101_multi_1,314,5,14,High SB
9072,17,SOUTH SAN FRANCISCO,REG (C-LCL) 101_multi_1,319,5,19,High SB
9073,20,SAN BRUNO,REG (C-LCL) 101_multi_1,322,5,22,High SB
9074,24,MILLBRAE,REG (C-LCL) 101_multi_1,326,5,26,High SB
9075,29,BURLINGAME,REG (C-LCL) 101_multi_1,330,5,30,High SB
9096,31,SAN MATEO,REG (C-LCL) 101_multi_1,333,5,33,High SB
9097,33,HAYWARD PARK,REG (C-LCL) 101_multi_1,336,5,36,High SB


In [62]:
df_final.to_csv('../data/parsed_data/schedules/future_schedules_cleaned.csv', index=False)